Using: 

https://medium.com/future-vision/google-maps-in-python-part-2-393f96196eaf

https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf

In [1]:
import os
import pandas as pd
import gmaps
from datetime import datetime
import googlemaps
import re

In [2]:
# create function to read in api key from txt file
def read_in_google_api_key(file_directory):
    api_key = pd.read_csv("gmaps_api_key.txt", header = None).iloc[0, 0]
    return api_key

In [3]:
def return_cityState_from_longitudeLatitude(long, lat):
    return googlemaps_client.reverse_geocode((long, lat))[0]['formatted_address']

### Setup Key

In [4]:
# read in api key from txt file
google_api_key = read_in_google_api_key(os.getcwd())

In [5]:
# setup api key for gmaps api
gmaps.configure(api_key=google_api_key)

In [6]:
# setup api key for GoogleMaps api
googlemaps_client = googlemaps.Client(key = google_api_key)

### Setup two example locations

In [7]:
location1 = (googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lng'])

In [8]:
location2 = (googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lng'])

### Trip Logistics

In [9]:
def create_optimal_trip_directions(googlemaps_client, location1, location2):
    direction_route_info = googlemaps_client.directions(location1, location2, mode = "driving")[0]
    
    # distance of entire trip
    trip_distance = direction_route_info['legs'][0]['distance']['text']
    print("trip distance: "+str(trip_distance))
    # duration of entire trip
    trip_duration = direction_route_info['legs'][0]['duration']['text']
    print("trip duration: "+str(trip_duration))
    return direction_route_info, trip_distance, trip_duration

In [10]:
direction_route_info, trip_distance, trip_duration = create_optimal_trip_directions(googlemaps_client, location1, location2)

trip distance: 451 mi
trip duration: 7 hours 3 mins


### Stops/Waypoints

In [15]:
df_tripDirections = pd.DataFrame(columns = ['start_location_lat', 'start_location_lng', 'end_location_lat', 'end_location_lng', 
                                            'distance_value', 'distance_text', 'duration_value', 'duration_text'])
i = 0
#these are the steps of the trip, my goal is to show the weather at each of them, (every other?)
for direction in direction_route_info['legs'][0]['steps']:

    df_tripDirections.loc[i] = [direction['start_location']['lat'], direction['start_location']['lng'], 
                                direction['end_location']['lat'], direction['end_location']['lng'], 
                                direction['distance']['value'], direction['distance']['text'], 
                                direction['duration']['value'], direction['duration']['text']]
    i += 1

In [16]:
def return_weather_checkpoint_approval(df_tripDirections_mileStops, distance_between_weather_waypoints):
    """
    This function iteratively checks the distance of each change in direction and 
    finds optimal points to put a weather waypoint based on the distance_between_weather_waypoints (in miles) parameter
    """
    df_tripDirections_mileStops.reset_index(drop=True, inplace=True)
    cummulative_adding_value = 0
    df_tripDirections_mileStops['weather_checkpoint'] = 0
    for i in range(1, len(df_tripDirections_mileStops)):
        current_row_value = df_tripDirections_mileStops.loc[i, 'distance_number']
        prev_row_value = df_tripDirections_mileStops.loc[i-1, 'distance_number']
        
        if current_row_value - prev_row_value >= distance_between_weather_waypoints:
            df_tripDirections_mileStops.loc[i, 'weather_checkpoint'] = 1
        elif cummulative_adding_value >= distance_between_weather_waypoints:
            df_tripDirections_mileStops.loc[i, 'weather_checkpoint'] = 1
            cummulative_adding_value = 0
        else:
            cummulative_adding_value += current_row_value
    
    return df_tripDirections_mileStops

In [27]:
df_tripDirections['distance_number'] = df_tripDirections['distance_text'].str.replace(r"[a-zA-Z]", '').astype('float')
df_tripDirections['distance_type'] = df_tripDirections['distance_text'].str.replace('\d+', '').str.replace('. ',"").str.strip().astype("string")
df_tripDirections_mileStops = df_tripDirections.query("distance_type == 'mi'")

df_tripWaypoint_locations = return_weather_checkpoint_approval(df_tripDirections_mileStops, 30)

C:\Users\David\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\David\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [32]:
#df_tripWaypoint_locations = df_tripDirections[df_tripDirections['distance_value'] >= 40000]
df_tripWaypoint_locations['end_location_text'] = df_tripWaypoint_locations.apply(lambda x: return_cityState_from_longitudeLatitude(x['end_location_lat'], x['end_location_lng']), axis = 1)
df_tripWaypoint_locations

C:\Users\David\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,start_location_lat,start_location_lng,end_location_lat,end_location_lng,distance_value,distance_text,duration_value,duration_text,distance_number,distance_type,weather_checkpoint,end_location_text
0,40.484250,-88.993770,40.536407,-88.995884,5807,3.6 mi,530,9 mins,3.6,mi,0,"2200 N Main St, Normal, IL 61761, USA"
1,40.536407,-88.995884,41.479901,-88.197113,136879,85.1 mi,4760,1 hour 19 mins,85.1,mi,1,"22140 SW Frontage Rd, Joliet, IL 60404, USA"
2,41.479901,-88.197113,41.536368,-87.968793,21791,13.5 mi,838,14 mins,13.5,mi,0,"130 Beverly Blvd, New Lenox, IL 60451, USA"
3,41.536368,-87.968793,41.562236,-87.751076,18809,11.7 mi,639,11 mins,11.7,mi,0,"19 Central Ave, Country Club Hills, IL 60478, USA"
4,41.562236,-87.751076,41.563986,-87.745758,484,0.3 mi,17,1 min,0.3,mi,0,"I-80, Country Club Hills, IL 60478, USA"
5,41.563986,-87.745758,41.582801,-87.686019,5557,3.5 mi,191,3 mins,3.5,mi,0,"I-80, Hazel Crest, IL 60429, USA"
6,41.582559,-87.684496,41.584602,-87.234983,38023,23.6 mi,1328,22 mins,23.6,mi,0,"I-94, Lake Station, IN 46405, USA"
7,41.584602,-87.234983,41.386756,-82.178565,447806,278 mi,14971,4 hours 10 mins,278.0,mi,1,"I-90, Elyria, OH 44035, USA"
8,41.386756,-82.178565,41.474452,-81.723184,43405,27.0 mi,1531,26 mins,27.0,mi,1,"2100 W 53rd St, Cleveland, OH 44102, USA"
9,41.474452,-81.723184,41.473876,-81.705644,1485,0.9 mi,53,1 min,0.9,mi,0,"3323 Wade Ave, Cleveland, OH 44113, USA"


In [ ]:
df_weathdf_tripWaypoint_locations

In [33]:
def create_custom_markers_from_directions(df_tripWaypoint_locations):
    list_custom_waypoint_dictionaries = []
    for i in range(len(df_tripWaypoint_locations)):
        temp_waypoint_location = df_tripWaypoint_locations.iloc[i, :]
        dict_custom_waypoint_info = {"name": temp_waypoint_location['end_location_text'],
                                    "location": (temp_waypoint_location['end_location_lat'], temp_waypoint_location['end_location_lng']),
                                    "weather_degrees": "TBD",
                                    "weather_info": "TBD"}
        list_custom_waypoint_dictionaries.append(dict_custom_waypoint_info)
    
    info_box_template = """
        <dl>
        <dt>Location</dt><dd>{name}</dd>
        <dt>Weather (Degrees)</dt><dd>{weather_degrees}</dd>
        <dt>Weather (Info)</dt><dd>{weather_info}</dd>
        </dl>
    """
    custom_waypoint_locations = [waypoint['location'] for waypoint in list_custom_waypoint_dictionaries]
    custom_waypoint_info = [info_box_template.format(**waypoint) for waypoint in list_custom_waypoint_dictionaries]
    
    return custom_waypoint_locations, custom_waypoint_info

In [34]:
custom_waypoint_locations, custom_waypoint_info = create_custom_markers_from_directions(df_tripWaypoint_locations)

### Plot

In [ ]:
#Create the map
fig = gmaps.figure()
#create the layer
layer = gmaps.directions.Directions(location1, location2, mode='car')
#Add the layer
fig.add_layer(layer)
custom_markers = gmaps.marker_layer(custom_waypoint_locations, info_box_content=custom_waypoint_info)
fig.add_layer(custom_markers)
fig